In [1]:
import requests
from urllib.request import urlopen
from urllib.parse import urlencode,unquote,quote_plus
import urllib
import lxml
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [2]:
# 법령 XML 상세조회 (시행기준. target=eflaw)

#기본 url
url_base = " http://www.law.go.kr/DRF/lawService.do"

#조회 url 세부 설정
user_id = 'bmsong' #open API ID (bmsong@kau.ac.kr의 아이디 부분)
target = "eflaw" #법령 조회시
MST = "230163" #ID 또는 MST #건설기술진흥법 MST 230163
#ID = "001807" 
            #ID 건설기술진흥법 법령ID 001807 (설명에는 ID로 조회하면 현행법령이 조회된다는데, 
            #target을 eflaw로 하면 ID로 요청해도, MST로 요청해도 같은 것 같다.
Type = "XML" #출력 형태 : HTML 또는 XML

url_sub = "?" +\
        "OC=" + user_id +\
        "&target=" + target +\
        "&MST=" + MST +\
        "&type=" + Type

#최종 url
url = url_base + url_sub

print(url)

response = requests.get(url)

#(1) response.text 사용하는 방법
#soup = BeautifulSoup(response.text.encode('utf-8'), 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)

#(2) response.content 사용하는 방법
soup = BeautifulSoup(response.content, 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)


# soup

 http://www.law.go.kr/DRF/lawService.do?OC=bmsong&target=eflaw&MST=230163&type=XML


In [3]:
lc_basic_df = pd.DataFrame() #DataFrame for Law Content 

data = soup.find('법령ID') #MST 정보
lc_basic_df['법령ID'] = [data.get_text()]

data = soup.find('법종구분') 
lc_basic_df['법종구분']= [data.get_text()]

data = soup.find('법령명_한글') 
lc_basic_df['법령명']= [data.get_text()]

data = soup.find('시행일자')
lc_basic_df['시행일자'] = [data.get_text()]

data = soup.find('공포일자') 
lc_basic_df['공포일자']= [data.get_text()]

data = soup.find('소관부처') 
lc_basic_df['소관부처']= [data.get_text()]

lc_basic_df

,법령ID,법종구분,법령명,시행일자,공포일자,소관부처
0,001807,법률,건설기술 진흥법,20210617,20210316,국토교통부


In [4]:
data = soup.find_all('조문단위')
df_info = pd.DataFrame()
info = []
for i in data:
    detail = {}
    detail["index"] = np.nan
    detail["조문제목"] = np.nan
    detail["조문내용"] = np.nan
    detail["항문내용"] = np.nan
    detail["호문내용"] = np.nan
    if i.find("조문여부").get_text() == "조문":
        조문제목 = i.find("조문내용").get_text().strip().split(")")[0]+")"
        detail["조문제목"] = 조문제목
        if i.find("조문내용"):
            조문내용 = i.find("조문내용").get_text().strip()[len(조문제목):-1].strip()
            detail["조문내용"] = 조문내용
        
        df_info = df_info.append(detail,ignore_index = True)
        
        if i.find("항"):
            for j in i.find_all("항"):
                if j.find("항내용"):
                    detail["항문내용"] = j.항내용.get_text()
                    df_info = df_info.append(detail,ignore_index = True)
                if j.find("호내용"):
                    for k in j.find_all("호내용"):
                        detail["호문내용"] = k.get_text().strip()
                        df_info = df_info.append(detail,ignore_index = True)
                detail["호문내용"] = np.nan
df_info = df_info[["조문제목","조문내용","항문내용","호문내용"]]
df_info

,조문제목,조문내용,항문내용,호문내용
0,제1조(목적),이 법은 건설기술의 연구ㆍ개발을 촉진하여 건설기술 수준을 향상시키고 이를 바탕으로 ...,NaN,NaN
1,제2조(정의),"이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2015.5.18, 2015....",NaN,NaN
2,제2조(정의),"이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2015.5.18, 2015....",NaN,"1. ""건설공사""란 「건설산업기본법」 제2조제4호에 따른 건설공사를 말한다."
3,제2조(정의),"이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2015.5.18, 2015....",NaN,"2. ""건설기술""이란 다음 각 목의 사항에 관한 기술을 말한다. 다만, 「산업안전보..."
4,제2조(정의),"이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2015.5.18, 2015....",NaN,"3. ""건설엔지니어링""이란 다른 사람의 위탁을 받아 건설기술에 관한 업무를 수행하는..."
...,...,...,...,...
704,제91조(과태료),,③ 다음 각 호의 어느 하나에 해당하는 자에게는 300만원 이하의 과태료를 부과...,14. 제62조제3항ㆍ제5항 및 제8항에 따른 서류를 제출하지 아니하거나 거짓으로 ...
705,제91조(과태료),,③ 다음 각 호의 어느 하나에 해당하는 자에게는 300만원 이하의 과태료를 부과...,15. 제62조제18항에 따른 설계의 안전성 검토결과를 제출하지 아니하거나 거짓으로...
706,제91조(과태료),,③ 다음 각 호의 어느 하나에 해당하는 자에게는 300만원 이하의 과태료를 부과...,16. 제67조제1항에 따른 건설사고 발생사실을 발주청 및 인ㆍ허가기관에 통보하지 ...
707,제91조(과태료),,④ 제1항부터 제3항까지에 따른 과태료는 대통령령으로 정하는 바에 따라 국토교통...,NaN


In [5]:
df_info.to_excel('건설기술진흥법.xlsx',index = False)